Preliminary data exploration

In [2]:
%matplotlib inline
import numpy as np
import pandas as pd
import matplotlib.pylab as plt
import seaborn as sns
import dmba
from pathlib import Path

from sklearn import preprocessing
from sklearn.tree import DecisionTreeClassifier, DecisionTreeRegressor
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.linear_model import LogisticRegression, LogisticRegressionCV
from sklearn.neighbors import NearestNeighbors, KNeighborsClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LinearRegression
import scikitplot as skplt

from dmba import regressionSummary
from dmba import adjusted_r2_score, AIC_score, BIC_score
from dmba import backward_elimination, forward_selection, stepwise_selection

from dmba import plotDecisionTree, classificationSummary, regressionSummary


from dmba import gainsChart, liftChart
#from dmba.metric import AIC_score
#import dmba

import warnings
warnings.filterwarnings('ignore')
warnings.simplefilter(action='ignore', category=FutureWarning)

In [30]:
movies_df = pd.read_csv('movies_metadata.csv')
movies_df.sample(5)

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
3775,False,NaN,0,"[{'id': 80, 'name': 'Crime'}, {'id': 18, 'name...",NaN,51179,tt0191037,en,Circus,"Bruno, a sadistic criminal, wants clever con m...",...,2000-05-05,0.0,95.0,"[{'iso_639_1': 'es', 'name': 'Español'}, {'iso...",Released,NaN,Circus,False,4.3,4.0
28816,False,NaN,0,"[{'id': 18, 'name': 'Drama'}, {'id': 12, 'name...",NaN,38198,tt0095386,fr,Itinéraire d'un enfant gâté,"A foundling, raised in the circus, Sam Lion be...",...,1988-11-30,0.0,135.0,"[{'iso_639_1': 'fr', 'name': 'Français'}]",Released,NaN,Itinerary of a Spoiled Child,False,7.6,23.0
12192,False,NaN,0,"[{'id': 28, 'name': 'Action'}, {'id': 18, 'nam...",NaN,48267,tt0065391,de,Der amerikanische Soldat,Ricky returns to Munich from Vietnam and is pr...,...,1970-10-09,0.0,80.0,"[{'iso_639_1': 'de', 'name': 'Deutsch'}]",Released,NaN,The American Soldier,False,8.0,2.0
9383,False,"{'id': 3963, 'name': 'The Crimson Rivers Colle...",0,"[{'id': 28, 'name': 'Action'}, {'id': 53, 'nam...",NaN,60672,tt0337103,fr,Les Rivières pourpres 2 : Les Anges de l'apoca...,"Jean Reno, Benoit Magimel and Christopher Lee ...",...,2004-02-18,0.0,97.0,"[{'iso_639_1': 'de', 'name': 'Deutsch'}, {'iso...",Released,An unholy crime... An unstoppable killer... Ca...,Crimson Rivers II: Angels of the Apocalypse,False,6.0,166.0
494,False,NaN,0,"[{'id': 35, 'name': 'Comedy'}, {'id': 10749, '...",NaN,31911,tt0107613,en,Mr. Wonderful,Electrician Gus gets the chance to fulfill a c...,...,1993-10-15,0.0,98.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Sometimes love is a stranger. And sometimes it...,Mr. Wonderful,False,5.5,11.0


In [10]:
keywords_df = pd.read_csv('keywords.csv')
keywords_df.sample(5)

,id,keywords
36626,211220,"[{'id': 1329, 'name': 'canada'}, {'id': 9826, ..."
39142,79594,[]
24749,91343,"[{'id': 291, 'name': 'circus'}, {'id': 604, 'n..."
14210,76636,"[{'id': 2496, 'name': 'impotence'}, {'id': 675..."
18994,94363,"[{'id': 160309, 'name': 'child abandonment'}, ..."


In [11]:
ratings_df = pd.read_csv('ratings.csv')
ratings_df.sample(5)

,userId,movieId,rating,timestamp
12100309,125485,11,3.5,1218792681
23762087,246661,5502,4.0,1032016867
7764266,80015,23,3.5,1420204388
22936010,238187,1201,3.0,999793222
9886859,101869,64839,4.5,1323288679


In [12]:
ratings_df.shape

(26024289, 4)

In [13]:
ratings_small_df = pd.read_csv('ratings_small.csv')
ratings_small_df.sample(5)

,userId,movieId,rating,timestamp
84514,564,3548,3.0,974713786
76548,529,31522,4.0,1264719725
65843,468,1069,3.0,1296194882
47682,350,4989,2.0,1027356424
6555,36,261,4.0,847057120


In [15]:
ratings_small_df.shape

(100004, 4)

In [16]:
links_small_df = pd.read_csv('links_small.csv')
links_small_df.sample(5)

,movieId,imdbId,tmdbId
7137,65230,822832,14306.0
3153,3945,259974,20455.0
7990,92420,1706593,76726.0
7189,67267,862846,13090.0
8699,115534,1204977,242512.0


In [19]:
movies_df.isna().sum()

adult                        0
belongs_to_collection    40972
budget                       0
genres                       0
homepage                 37684
id                           0
imdb_id                     17
original_language           11
original_title               0
overview                   954
popularity                   5
poster_path                386
production_companies         3
production_countries         3
release_date                87
revenue                      6
runtime                    263
spoken_languages             6
status                      87
tagline                  25054
title                        6
video                        6
vote_average                 6
vote_count                   6
dtype: int64

In [21]:
keywords_df.isna().sum()

id          0
keywords    0
dtype: int64

In [18]:
ratings_small_df.isna().sum()

userId       0
movieId      0
rating       0
timestamp    0
dtype: int64

In [17]:
ratings_df.isna().sum()

userId       0
movieId      0
rating       0
timestamp    0
dtype: int64

In [4]:
ratings_df = pd.read_csv('ratings.csv')
ratings_df.sample(5)

,userId,movieId,rating,timestamp
60740,391,4235,2.0,1030943882
19322,125,8665,3.0,1474415015
20065,132,2396,3.0,1157921317
86741,561,2355,3.0,1491095398
10664,68,1882,3.0,1158535346


In [7]:
tags_df = pd.read_csv('tags.csv')
tags_df.sample(5)

,userId,movieId,tag,timestamp
1605,474,2324,Holocaust,1137181119
2740,477,69757,quirky,1279956145
1836,474,3735,police,1138032794
2778,513,750,Slim Pickens,1159980456
3184,567,99917,dreamlike,1525286649


In [8]:
links_df = pd.read_csv('links.csv')
links_df.sample(5)

,movieId,imdbId,tmdbId
9327,160422,43147,68309.0
1521,2053,104437,11158.0
9422,165947,1542768,339396.0
2546,3408,195685,462.0
2989,4003,96094,12714.0


In [22]:
movies_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 45466 entries, 0 to 45465
Data columns (total 24 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   adult                  45466 non-null  object 
 1   belongs_to_collection  4494 non-null   object 
 2   budget                 45466 non-null  object 
 3   genres                 45466 non-null  object 
 4   homepage               7782 non-null   object 
 5   id                     45466 non-null  object 
 6   imdb_id                45449 non-null  object 
 7   original_language      45455 non-null  object 
 8   original_title         45466 non-null  object 
 9   overview               44512 non-null  object 
 10  popularity             45461 non-null  object 
 11  poster_path            45080 non-null  object 
 12  production_companies   45463 non-null  object 
 13  production_countries   45463 non-null  object 
 14  release_date           45379 non-null  object 
 15  re

parse dataset movies_df:

In [23]:
movies_df.head()

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0


In [24]:
movies_df['genres'].value_counts()

[{'id': 18, 'name': 'Drama'}]                                                                                                         5000
[{'id': 35, 'name': 'Comedy'}]                                                                                                        3621
[{'id': 99, 'name': 'Documentary'}]                                                                                                   2723
[]                                                                                                                                    2442
[{'id': 18, 'name': 'Drama'}, {'id': 10749, 'name': 'Romance'}]                                                                       1301
                                                                                                                                      ... 
[{'id': 28, 'name': 'Action'}, {'id': 18, 'name': 'Drama'}, {'id': 35, 'name': 'Comedy'}, {'id': 99, 'name': 'Documentary'}]             1
[{'id': 10752, 'name': 'War

In [31]:
# Comedy, War, Action, Crime, History, 
# Documentary, Thriller, Drama,
# Family, Animation, Romance, Horror,


# Find rows with different genres and generate
# columns based on these features:

import re

ids = movies_df.genres.str.contains('Comedy', flags = re.IGNORECASE, regex = True, na = False)
movies_df.insert(0,'Comedy_genre',0)
movies_df['Comedy_genre'].iloc[ids] = 1

ids_war = movies_df.genres.str.contains('War', flags = re.IGNORECASE, regex = True, na = False)
movies_df.insert(0,'War_genre',0)
movies_df['War_genre'].iloc[ids_war] = 1

ids_crime = movies_df.genres.str.contains('Crime', flags = re.IGNORECASE, regex = True, na = False)
movies_df.insert(0,'Crime_genre',0)
movies_df['Crime_genre'].iloc[ids_crime] = 1

ids_action = movies_df.genres.str.contains('Action', flags = re.IGNORECASE, regex = True, na = False)
movies_df.insert(0,'Action_genre',0)
movies_df['Action_genre'].iloc[ids_action] = 1

ids_documentary = movies_df.genres.str.contains('Documentary', flags = re.IGNORECASE, regex = True, na = False)
movies_df.insert(0,'Documentary_genre',0)
movies_df['Documentary_genre'].iloc[ids_documentary] = 1

ids_thriller = movies_df.genres.str.contains('Thriller', flags = re.IGNORECASE, regex = True, na = False)
movies_df.insert(0,'Thriller_genre',0)
movies_df['Thriller_genre'].iloc[ids_thriller] = 1

ids_drama = movies_df.genres.str.contains('Drama', flags = re.IGNORECASE, regex = True, na = False)
movies_df.insert(0,'Drama_genre',0)
movies_df['Drama_genre'].iloc[ids_drama] = 1

ids_family = movies_df.genres.str.contains('Family', flags = re.IGNORECASE, regex = True, na = False)
movies_df.insert(0,'Family_genre',0)
movies_df['Family_genre'].iloc[ids_family] = 1

ids_animation = movies_df.genres.str.contains('Animation', flags = re.IGNORECASE, regex = True, na = False)
movies_df.insert(0,'Animation_genre',0)
movies_df['Animation_genre'].iloc[ids_animation] = 1

ids_romance = movies_df.genres.str.contains('Romance', flags = re.IGNORECASE, regex = True, na = False)
movies_df.insert(0,'Romance_genre',0)
movies_df['Romance_genre'].iloc[ids_romance] = 1

ids_horror = movies_df.genres.str.contains('Horror', flags = re.IGNORECASE, regex = True, na = False)
movies_df.insert(0,'Horror_genre',0)
movies_df['Horror_genre'].iloc[ids_horror] = 1


In [32]:
movies_df

,Horror_genre,Romance_genre,Animation_genre,Family_genre,Drama_genre,Thriller_genre,Documentary_genre,Action_genre,Crime_genre,War_genre,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,0,0,1,1,0,0,0,0,0,0,...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,0,0,0,1,0,0,0,0,0,0,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,0,1,0,0,0,0,0,0,0,0,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,0,1,0,0,1,0,0,0,0,0,...,1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,0,0,0,0,0,0,0,0,0,0,...,1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
45461,0,0,0,1,1,0,0,0,0,0,...,NaN,0.0,90.0,"[{'iso_639_1': 'fa', 'name': 'فارسی'}]",Released,Rising and falling between a man and woman,Subdue,False,4.0,1.0
45462,0,0,0,0,1,0,0,0,0,0,...,2011-11-17,0.0,360.0,"[{'iso_639_1': 'tl', 'name': ''}]",Released,NaN,Century of Birthing,False,9.0,3.0
45463,0,0,0,0,1,1,0,1,0,0,...,2003-08-01,0.0,90.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,A deadly game of wits.,Betrayal,False,3.8,6.0
45464,0,0,0,0,0,0,0,0,0,0,...,1917-10-21,0.0,87.0,[],Released,NaN,Satan Triumphant,False,0.0,0.0


In [ ]:
date=[]
time=[]

def conv_datetime(x):
    if pd.isnull(x): 
      # if there are nulls, exit out and return the null value
        return x
    spt = x.split('}, ') 
    date.append(spt[0]) 

    if len(spt) > 1: 
      # if there is more than 1 value in the array spt
      # that was separated by spaces,
        time.append(spt[1]) 

    return date, time

In [ ]:
# Create predictor based on time of day
# of when the taxi begins its ride:

timeOfDay=[]
#time=[]

def conv_timeofday(x):
    if pd.isnull(x): 
      # if there are nulls, exit out and return the null value
        return x
    hour = x.split(':') 
    hour_int = int(hour[0])
    if hour_int <= 4:
        timeOfDay.append('Early')
    elif hour_int <= 10:
        timeOfDay.append('Morning')
    elif hour_int <= 15:
        timeOfDay.append('Midday')
    elif hour_int <= 19:
        timeOfDay.append('Evening')
    else:
        timeOfDay.append('Night')

    return timeOfDay

In [ ]:
# Generate binary features for the vehicle_mobile_id
# and from_area_id classes of interest:

area293=taxi_df.groupby(['from_area_id']).get_group(293.0).index

vehicle89=taxi_df.groupby(['vehicle_model_id']).get_group(89).index

In [ ]:
taxi_df.insert(0,'area_293',0)
taxi_df.insert(0,'vehicle_89',0)

combine links to movie_df through 'movie_id' column:

combine movies_df to ratings_df through 'movie_id' column:

combine keywords to ratings_df through 'user_id' column: